# Import Python Packages

# Import Python packages 
import datetime
import pandas as pd
import os
import glob
import psycopg2

# Connection Set up 

## DB Params

In [ ]:
DB_ENDPOINT = ""
DEFAULT_DB = 'postgres'
DB = ''
DB_USER = ''
DB_PASSWORD = ''
DB_PORT = '5432'

## Create a connection to Default Database

In [ ]:
try: 
    conn = psycopg2.connect(
                        host=DB_ENDPOINT,
                        port=DB_PORT,
                        dbname=DEFAULT_DB,
                        user=DB_USER,
                        password=DB_PASSWORD)
    print("Connection established")
    conn.set_session(autocommit=True)
    Print("Connection is in Auto Commit")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Default Postgres database")
    print(e)

## Create Cursor

In [ ]:
try: 
    cursor = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)

## Create New DB and reset connection 

In [ ]:
query = "DROP DATABASE IF EXISTS %s"
try: 
    cursor.execute(query, (DB))
    query = "CREATE DATABASE %s"
    cursor.execute(query, (DB))
except psycopg2.Error as e: 
    print("Error: Issue creating DataBase")
    print (e)

### Reset connection to the Desired DB

In [ ]:
cursor.close()
conn.close()
try: 
    conn = psycopg2.connect(
                        host=DB_ENDPOINT,
                        port=DB_PORT,
                        dbname=DB,
                        user=DB_USER,
                        password=DB_PASSWORD)
    print("Connection established")
    conn.set_session(autocommit=True)
    Print("Connection is in Auto Commit")
except psycopg2.Error as e: 
    print(f"Error: Could not make connection to the {DB} database")
    print(e)

# Populating Cleaned Data 

## Automation Functions

In [ ]:
# Helper functions
def table_recreate(cursor, tableName: str, tableFields: str ):
     """ Takes the Connection , Table name and create Fields, will drop the table if exists and create it again with the desired fields Ex : table_recreate(cursor, payments, "(sessionId INT,
                    itemInSession INT,
                    artist TEXT,
                    song TEXT,
                    length FLOAT, 
                    PRIMARY KEY (sessionId, itemInSession))")  """
    try:
        
        cursor.execute("DROP TABLE IF EXISTS {0};".format(tableName))
        query = f"CREATE TABLE IF NOT EXISTS {tableName} "
        query = query + tableFields
        cursor.execute(query)
        print("Finished creating table {0}".format(tableName))
    except psycopg2.Error as e: 
    print(f"Error: Couldn't recreate the table: {tableName}, something went wrong")
    print(e)

def populate_table(c, filename, tablename):
    f = open(filename, 'r')
    try:
        cursor.copy_from(f, tablename, sep=",", null = "")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
    print("Finished populating {0}".format(tablename))
    def generate_header(signal_header_dict: dict) -> pandas.DataFrame:
    """ Takes the header_attributes associated with The signal and Return a Pandas Data Frame with the format
        of { key : [value],} for the appropriate  Signal"""

    header_dict = {key: [value] for (key, value) in signal_header_dict.items()}
    return pandas.DataFrame(header_dict)
